<a href="https://colab.research.google.com/github/sabinahartnett/Data-and-Code/blob/main/Question3Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sabina Hartnett 
### Question 3 - Implementing a Reproducible Machine Learning Pipeline
with some code acquired from John Snow Labs (https://nlp.johnsnowlabs.com/2021/01/15/analyze_sentimentdl_use_imdb_en.html) 

# Spark NLP 
### Had issues loading in the PySpark 'spark = SparkSession.builder.master("local[*]").getOrCreate()' from the Local/Colab Spark Setup given to us since it then wouldn't allow me to use Spark NLP... so using this here since it is working

In [ ]:
''' 
# NOTE: this code was causing errors later with my code... so commented out
%%bash
apt-get update
apt-get install -y openjdk-8-jdk-headless -qq > /dev/null

wget -q "https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz" > /dev/null
tar -xvf spark-3.1.1-bin-hadoop2.7.tgz > /dev/null

pip install pyspark findspark
# Set path to Spark
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"

# Find Spark so that we can access session within our notebook
import findspark
findspark.init()

# Start SparkSession on all available cores
from pyspark.sql import SparkSession
#changed in the builder using code from: https://nlp.johnsnowlabs.com/docs/en/install
spark = SparkSession.builder.appName("Spark NLP").master("local[*]").config("spark.driver.memory","16G").config("spark.driver.maxResultSize", "0").config("spark.kryoserializer.buffer.max", "2000M").config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:3.0.3").getOrCreate()
'''

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/quick_start_google_colab.ipynb)

In [3]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2021-05-31 00:46:03--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2021-05-31 00:46:03--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1608 (1.6K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.57K  --.-KB/s    in 0s      

2021-05-31 00:46:03 (28.3 MB/s) - written to stdout [1608/1608]

setup Cola

We will first set up the runtime environment and then load pretrained Entity Recognition model and Sentiment analysis model and give it a quick test. 

In [4]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

Spark NLP version: 3.0.3
Apache Spark version: 3.0.2


In [5]:
from sparknlp.pretrained import PretrainedPipeline 

Let's use Spark NLP pre-trained pipeline for `named entity recognition`

In [5]:
pipeline_ent = PretrainedPipeline('recognize_entities_dl', 'en')

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]


In [8]:
result_ent = pipeline_ent.annotate('President Biden represented Delaware for 36 years in the U.S. Senate before becoming the 47th Vice President of the United States.') 

In [9]:
print(result_ent['entities'])

['Biden', 'Delaware', 'U.S', 'Senate', 'United States']


Let's use Spark NLP pre-trained pipeline for `sentiment` analysis

In [4]:
pipeline = PretrainedPipeline('analyze_sentimentdl_glove_imdb', 'en')

analyze_sentimentdl_glove_imdb download started this may take some time.
Approx size to download 155.3 MB
[OK!]


In [50]:
pipeline.annotate('President Biden represented Delaware for 36 years in the U.S. Senate before becoming the 47th Vice President of the United States.')['sentiment']

['pos']

In [7]:
print(result['sentiment'])

['pos']


In [6]:
!pip install wget
import wget
wget.download('https://s3.amazonaws.com/amazon-reviews-pds/tsv/sample_us.tsv', 'sample_data/sample_us.tsv')
# Read TSV file from default data download directory in Colab
data = spark.read.csv('sample_data/sample_us.tsv',
                      sep="\t",
                      header=True,
                      inferSchema=True)

  Created wheel for wget: filename=wget-3.2-cp37-none-any.whl size=9681 sha256=d4fcdeb8ea2b4387a94bdf4807fd5ade8112e8a3c4884c758580012a90bb6eb5
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [13]:
data.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [14]:
(data.groupBy('star_rating')
     .sum('total_votes')
     .sort('star_rating', ascending=False)
     .show()
)

+-----------+----------------+
|star_rating|sum(total_votes)|
+-----------+----------------+
|          5|              13|
|          4|               3|
|          3|               8|
|          2|               2|
|          1|               8|
+-----------+----------------+



In [7]:
from pyspark.sql.functions import *

In [8]:
data = (data.withColumn('star_rating', col('star_rating').cast('int'))
            .withColumn('total_votes', col('total_votes').cast('int'))
            .withColumn('helpful_votes', col('helpful_votes').cast('int'))
            )
data.printSchema

<bound method DataFrame.printSchema of DataFrame[marketplace: string, customer_id: int, review_id: string, product_id: string, product_parent: int, product_title: string, product_category: string, star_rating: int, helpful_votes: int, total_votes: int, vine: string, verified_purchase: string, review_headline: string, review_body: string, review_date: string]>

In [9]:
# From code given
data = data.withColumn('good_review', (data.star_rating >= 4).cast("integer"))

(data.groupBy('good_review')
     .count()
     .show()
)

+-----------+-----+
|good_review|count|
+-----------+-----+
|          1|   37|
|          0|   12|
+-----------+-----+



## 3a. Engineering additional features
NOW to **add the additional variables**: I have calculated the average number of total votes (5.33), helpful votes (3.84) and the average rating (4.3) - these values were acquired using pyspark on the FULL dataset on midway (see other notebook for those calculations) so I have created three variables which ==1 when the respective column is above average and 0 otherwise, as well as a column which calculates the length of the review of that Amazon item (here toys but in the larger set books).

'*above_avg_total_votes*' was created to contextualize the number of total votes: while it may still be useful to investigate the relationship between the raw number of votes, this cutoff may be a simplified way to investigate (if there is) the relationship between the total number of votes and whether the star rating of the item is 'good' (>3).

'*above_avg_helpful_votes*' was created to contextualize the number of helpful votes: while it may still be useful to investigate the relationship between the raw number of votes, this cutoff may be a simplified way to investigate (if there is) the relationship between the number of helpful votes and whether the star rating of the item is 'good' (>3).

'*review_len*' Shows the length of the respective review. As I was investigating the data, a natural question emerged: 'do people write longer positive or longer negative reviews?' and is this value consistent and interpretable? This variable will help us investigate and understand these two questions

In [17]:
data = data.withColumn('above_avg_total_votes', (data.total_votes > 5.33).cast('integer'))
data = data.withColumn('above_avg_helpful_votes', (data.helpful_votes > 3.84).cast('integer'))

#the feature below will too similar to 'good_review'
#data = data.withColumn('above_avg_rating', (data.star_rating > 4.3).cast('integer'))
data = data.withColumn('review_length', length(col('review_body')))

# Engineering Text-Derived Features:
Here I have chosen to include the overall sentiment of the review and the number of entitites included in the review.

'*pos_sentiment*' == 1 when the review of the item is overall determined to be positive using sparknlp. This intuitively will help us in predicting the star rating of a product where positive sentiments will significantly likely predict a 'good' rating.

'*num_entities*' equals the number of entities identified in the review_body by the sparknlp pretrained pipeline. Ideally, it would be useful to identify whether the entity == the product or == amazon but that is room for further analysis. As is, this column can (abstractly) help us understand how directly/actively the reviewer is writing about their experience with the product and in the following steps we will investigate this features relationship to whether a review is 'good'.

In [42]:
#add sentiment analysis ('is positive sentiment' column)
from pyspark.sql import functions as f
#cant figure out how to contain this to a withColumn command - gives errors that you
#canot just pass a col
text_col = data.select(f.col("review_body").alias("text"))
# analyze sentiment
sntmt = pipeline.transform(text_col)
df_sntmnt = sntmt.selectExpr(["text", "sentiment[0].result"])
#extract just the output of the sentiment result into its own column 
df_sntmnt =df_sntmnt.withColumnRenamed("sentiment[0].result", "sentiment")
df_sntmnt = df_sntmnt.withColumn("pos_sentiment",
                           f.when((f.col("sentiment") == "pos"), 1).otherwise(0))

#join that column onto original DF
full_data = data.join(df_sntmnt, data.review_body == df_sntmnt.text, how = "inner")
full_data.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+-------------+-----------+--------------------+---------+-------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|above_avg_total_votes|above_avg_helpful_votes|above_avg_rating|review_length|good_review|                text|sentiment|pos_sentiment|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+----------

In [43]:
## and add entities
text_col = data.select(f.col("review_body").alias("text"))
# analyze sentiment
entities_col = pipeline_ent.transform(text_col)

#entities_df = entities_col.selectExpr(["text", "sentiment[0].result"])
#entities_col.select("entities.result").show(truncate=False)
entities_df = entities_col.withColumn('num_entities', size(col('entities.result')))
def_ents = entities_df.select('text','entities', 'num_entities')
#join that column onto original DF
full_data = full_data.join(def_ents, data.review_body == def_ents.text, how = "inner")
full_data = full_data.drop('text')
full_data.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+-------------+-----------+--------------------+---------+-------------+--------------------+--------------------+------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|above_avg_total_votes|above_avg_helpful_votes|above_avg_rating|review_length|good_review|                text|sentiment|pos_sentiment|                text|            entities|num_entities|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+-----------------

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+-------------+-----------+---------+-------------+--------------------+------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|above_avg_total_votes|above_avg_helpful_votes|above_avg_rating|review_length|good_review|sentiment|pos_sentiment|            entities|num_entities|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+-

In [36]:
data.show() #.groupBy('product_category').count().show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+-------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|above_avg_total_votes|above_avg_helpful_votes|above_avg_rating|review_length|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+-------------+
|         US|   18778586| RDIJS7QYB6XNR|B00EDBY7X8|     122952789|Monopoly Junior B...|            Toys|     

## 3b. Organize Transformers into Pipeline

In [51]:
'''
# This is the pipeline format I found online ... but is difficult to work with - for now I'll make a function
sentimentdl = PretrainedPipeline('analyze_sentimentdl_glove_imdb', 'en').setInputCols(["review_body"]).setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          sentimentdl
      ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

df = spark.createDataFrame(pd.DataFrame({"text":data.review_body}))
result = pipelineModel.transform(df)

result.show()
'''

analyze_sentimentdl_glove_imdb download started this may take some time.
Approx size to download 155.3 MB
[OK!]


AttributeError: ignored

In [13]:
from pyspark.sql import functions as f

In [14]:
def features_pipeline(data_df, text_col, non_text_features=True, sentiment=True, entities=True):
  ''' this can now be called on a pyspark dataframe and will add the respective features'''
  pipeline_ent = PretrainedPipeline('recognize_entities_dl', 'en')
  text_col = data_df.select(f.col(text_col).alias("text"))

  if non_text_features == True:
    data_df = data_df.withColumn('above_avg_total_votes', (data_df.total_votes > 5.33).cast('integer'))
    data_df = data_df.withColumn('above_avg_helpful_votes', (data_df.helpful_votes > 3.84).cast('integer'))
    data_df = data_df.withColumn('review_length', length(col('review_body')))
    full_data = data_df


  if sentiment == True:
    pipeline = PretrainedPipeline('analyze_sentimentdl_glove_imdb', 'en')
    # analyze sentiment
    sntmt = pipeline.transform(text_col)
    df_sntmnt = sntmt.selectExpr(["text", "sentiment[0].result"])
    #extract just the output of the sentiment result into its own column 
    df_sntmnt =df_sntmnt.withColumnRenamed("sentiment[0].result", "sentiment")
    df_sntmnt = df_sntmnt.withColumn("pos_sentiment",
                              f.when((f.col("sentiment") == "pos"), 1).otherwise(0))
    #join that column onto original DF
    full_data = data_df.join(df_sntmnt, data_df.review_body == df_sntmnt.text, how = "inner")
    full_data = full_data.drop('text')

  if entities == True:
    # analyze entities
    entities_col = pipeline_ent.transform(text_col)
    entities_df = entities_col.withColumn('num_entities', size(col('entities.result')))
    def_ents = entities_df.select('text','entities', 'num_entities')
    #join that column onto original DF
    full_data = full_data.join(def_ents, full_data.review_body == def_ents.text, how = "inner")
    full_data = full_data.drop('text')
  
  return full_data

In [15]:
output_df = features_pipeline(data, "review_body")

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]
analyze_sentimentdl_glove_imdb download started this may take some time.
Approx size to download 155.3 MB
[OK!]


In [16]:
output_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------+---------------------+-----------------------+-------------+---------+-------------+--------------------+------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|good_review|above_avg_total_votes|above_avg_helpful_votes|review_length|sentiment|pos_sentiment|            entities|num_entities|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+-----------+---------------------+-----------------------+-------------+---------

## 4 Finding Optimal Model Parameters
unfortunately I am still having issues with EMR (no word from AWS Education - although I followed up again on Friday)
Since this is so small I will run it here on the Colab

In [64]:
import numpy as np 
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml import Pipeline, PipelineModel

from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics


In [54]:
full_data.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+----------------+-------------+-----------+---------+-------------+--------------------+------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|above_avg_total_votes|above_avg_helpful_votes|above_avg_rating|review_length|good_review|sentiment|pos_sentiment|            entities|num_entities|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+---------------------+-----------------------+-

In [62]:
features = ['above_avg_total_votes', 'above_avg_helpful_votes', 'review_length', 'pos_sentiment', 'num_entities']

In [69]:
assembler = VectorAssembler(inputCols = features, outputCol = 'features', handleInvalid="keep")
new_df = assembler.transform(full_data)

In [70]:
#create training/test set 
train = new_df.sample(fraction = 0.7, seed = 0)
test = new_df.join(train, on = "review_id", how = "left_anti")

In [71]:
lr = LogisticRegression(featuresCol='features', labelCol='good_review')

In [72]:
model = lr.fit(train)
# Training Summary Data
trainSummary = model.summary
evalSummary = model.evaluate(test)

print("Training AUC: " + str(trainSummary.areaUnderROC))
print("Test AUC: ", str(evalSummary.areaUnderROC))

Training AUC: 0.9666666666666667
Test AUC:  0.7575757575757576


In [74]:
cv_train = train.selectExpr(["features", "good_review"])

In [75]:
cv_train =cv_train.withColumnRenamed("good_review", "label")

In [76]:
cv_test = test.selectExpr(["features", "good_review"])

In [77]:
cv_test =cv_test.withColumnRenamed("good_review", "label")

## Cross Validation

In [78]:
lr2 = LogisticRegression(maxIter=10)

paramGrid = ParamGridBuilder() \
    .addGrid(lr2.regParam, np.arange(0, 0.1, 0.01)) \
    .addGrid(lr2.elasticNetParam, [0, 1])\
    .build()

crossval = CrossValidator(estimator=lr2,
                          estimatorParamMaps=paramGrid,
                          evaluator=BinaryClassificationEvaluator(),
                          numFolds=5)  

# Run cross-validation, and choose the best set of parameters.
cvModel = crossval.fit(cv_train)

In [79]:
cvModel.avgMetrics

[0.8723809523809524,
 0.8723809523809524,
 0.8514285714285715,
 0.8514285714285715,
 0.900952380952381,
 0.900952380952381,
 0.900952380952381,
 0.8509523809523809,
 0.900952380952381,
 0.8128571428571427,
 0.900952380952381,
 0.8128571428571427,
 0.900952380952381,
 0.8128571428571427,
 0.900952380952381,
 0.8128571428571427,
 0.900952380952381,
 0.8128571428571427,
 0.900952380952381,
 0.8128571428571427]

In [83]:
BinaryClassificationEvaluator().evaluate(cvModel.transform(cv_test))

0.7878787878787878